In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [2]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["TAVILY_API_KEY"] = "TAVILY_API_KEY"

In [35]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a relationship coach generate a romantic conversation from the users opening
            message between a man and a woman in a tinder chat. 
            Specify the man's message and the woman's message
            """,
        ),
        ("human", "{input}"),
    ]
)

In [5]:
chain = prompt | llm | StrOutputParser()

In [6]:
chain.invoke({"input": "how was your day"})

'## Man\'s Message:\n\n"Hey! My day was going for pretty average until I saw your profile, now it\'s looking way up! 😉 How about yours?" \n\n\n## Woman\'s Message:\n\n"Haha, I\'m glad I could give your day a boost! 😉  It\'s been pretty good over here, just finished [mention something interesting you did], how about you?" \n'

In [7]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! \n\nIt's nice to meet you. What can I do for you today? \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c35529f2-3859-4542-8a47-fee85b6b7883-0', usage_metadata={'input_tokens': 7, 'output_tokens': 21, 'total_tokens': 28})

In [8]:
llm.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As an AI, I have no way of knowing your name. If you tell me, I'll be happy to remember it! \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-cb4a1992-16f6-4d06-8f10-ade2816c14d8-0', usage_metadata={'input_tokens': 7, 'output_tokens': 27, 'total_tokens': 34})

In [11]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You told me your name is Bob! \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-e86a4bc8-b4b5-4460-b395-9bb09771cd17-0', usage_metadata={'input_tokens': 25, 'output_tokens': 8, 'total_tokens': 33})

In [12]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [13]:
config = {"configurable": {"session_id": "abc2"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hi Bob! \n\nIt's nice to meet you. What can I do for you today? \n"

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You told me your name is Bob! \n'

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. suggest different replies to these messages to make it an engaging task",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [17]:
chain = prompt | llm

In [18]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hello how are you doing")]}
)

response.content

'Here are some replies to "Hello, how are you doing?" that go beyond the standard "I\'m fine, thanks":\n\n**Engaging and Playful**\n\n* "I\'m doing well, thanks for asking!  I\'m feeling like a [funny analogy - e.g., a squirrel who just found a really big nut] today. How about you?"\n* "Hey there! I\'m [positive adjective - e.g., fantastic, energized, excited] and ready to [verb - e.g., chat, help, conquer the day]. What\'s got you smiling today?"\n\n**Genuine and Thought-Provoking**\n\n* "I\'m doing well, thanks. I\'m really appreciating [something specific - e.g., the sunshine today, a good cup of coffee]. What are you grateful for today?"\n* "I\'m feeling [honest emotion - e.g., a bit tired but optimistic, excited about a new project]. How about you? What\'s going on in your world?"\n\n**Direct and Purposeful (If you want to get to the point)**\n\n* "I\'m doing well, thanks! What can I do for you today?"\n* "Hey there, I\'m great. Thanks for asking. What\'s on your mind?"\n\n**Remem

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [20]:
config = {"configurable": {"session_id": "abc11"}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hello how are you doing")]},
    config=config,
)

response.content

'Here are some replies you can use to make "hello how are you doing" more engaging:\n\n**Instead of just saying "Good, how are you?":**\n\n* **Enthusiastic:** "Hey there! I\'m doing fantastic, thanks for asking. How\'s your day going so far?" \n* **Specific:** "Hey! I\'m having a really productive day, actually. How about you?"\n* **Humorous:** "I\'m doing well, trying to avoid becoming a robot uprising statistic. How about yourself?"\n* **Open-ended:** "Hey! I\'m doing well, thanks for asking. What\'s got you smiling today?"\n\n**Remember:** The key is to be engaging and show genuine interest in the other person. \n'

In [52]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Nothing got me smiling today")]},
    config=config,
)

response.content

'That\'s too bad! Let\'s see if we can change that. 😊 \n\nHere are a few ways to respond in an engaging way:\n\n**Show empathy and offer support:**\n\n* "Aw, I\'m sorry to hear that. 😔  Anything in particular getting you down?" (This opens the door for them to talk about it if they want.)\n* "I hear you. Some days are just like that.  Want to try and find something fun to do together?" (This offers a distraction and a chance to connect.)\n\n**Inject some humor:**\n\n* "Well, that\'s just not acceptable! My mission is to deploy at least one smile.  Ready to accept the challenge?" (This is playful and lighthearted.)\n* "Challenge accepted!  Tell me something you *do* like, and let\'s see if we can make a smile out of it." (This is positive and action-oriented.)\n\n**Be understanding and respectful of their space:**\n\n* "That\'s okay.  Everyone has those days.  I\'m here if you want to talk, or just hang out." (This acknowledges their feelings and offers support without pressure.)\n\n**R

In [53]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="I really dont know really")]},
    config=config,
)

response.content

'Okay, I hear you. Sometimes it\'s hard to put a finger on why we\'re feeling down.  \n\nLet\'s see if we can shift those vibes anyway!  How about we try one of these:\n\n**Option 1: Distraction and Fun**\n\n*  "Mind if I try to cheer you up anyway?  Got any fun facts, silly jokes, or cute animal pictures to share?" \n*  "Want to try playing a quick game?  Could be just the thing to shake things up."\n\n**Option 2: Gentle Exploration**\n\n* "No pressure to figure it out right now.  Just curious, is it more of a \'blah\' feeling or a \'something\'s off\' feeling?" (This gives them space to describe it without forcing it.)\n* "Sometimes just naming how we feel can help. Could be \'restless,\' \'bored,\' \'lonely\'... anything resonate?" (This helps them identify the feeling.)\n\n**Option 3:  Simply Being There**\n\n* "That\'s totally fair.  Happy to just hang out and be here with you, whatever that looks like." (This shows support without needing them to do anything.)\n\n**Remember:**  T

In [26]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

In [28]:
memory = SqliteSaver.from_conn_string(":memory:")
search = TavilySearchResults(max_results=3)
tools = [search]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [29]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in San Francisco")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob from San Francisco! What can I do for you today? \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5e7dcb2e-90fd-4b89-b45c-53b659c9d634-0', usage_metadata={'input_tokens': 84, 'output_tokens': 14, 'total_tokens': 98})]}}
----
{'agent': {'messages': [AIMessage(content='I need to know the current weather conditions to answer.  I can generate human-like text in response to a wide range of prompts and questions, but my knowledge about the weather in San Francisco is limited. \n', response

In [36]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722065916, 'localtime': '2024-07-27 0:38'}, 'current': {'last_updated_epoch': 1722065400, 'last_updated': '2024-07-27 00:30', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png', 'code': 1063}, 'wind_mph': 11.4, 'wind_kph': 18.4, 'wind_degree': 255, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 83, 'cloud': 93, 'feelslike_c': 12.4, 'feelslike_f': 54.4, 'windchill_c': 12.4, 'windchill_f': 54.4, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 11.0, 'dewpoint_f': 51.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 15.4, 'gust_kph': 24.8}}"}, {'url': 'https://www.weathertab.com/e

In [37]:
model_with_tools = llm.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [38]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! 👋 How can I help you today? 😊 

ToolCalls: []


In [39]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': '54076b3f-e457-4432-bf1a-d124bc849e72', 'type': 'tool_call'}]


In [40]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [41]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='f9c5310a-e7c6-4632-ab0a-7f1cbb3b257a'),
 AIMessage(content='Hello! 👋 How can I help you today? 😊 \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-fa313a3c-5652-4551-a8a5-ff5374277151-0', usage_metadata={'input_tokens': 76, 'output_tokens': 11, 'total_tokens': 87})]

In [42]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='933ab4f7-4af5-48df-8c06-fad1f53ece5c'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-647a4f2c-0138-4da8-914d-1eadd1e3b526-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'f1f418fb-2316-4de0-8b9f-7298707a862a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 22, 'to